# Straight skeleton

## Определение
**Straight skeleton** — метод представления многоугольника его топологическим скелетом. То есть множество кривых, по которым проходят вершины многоугольника в процессе сжатия.  Интуитивное понимание: берем все стороны фигуры и начинаем с постоянной скоростью сдвигать "внутрь" параллельно себе. Вершины будут передвигаться по биссектрисам своих внутренних углов. Точка в скелете — место пересечения хотя бы двух несмежных сторон, или, по-другому, точка, в которой одна из сторон выродилась.

## Постановка задачи
Дан прямоугольник $P_n$, в котором $n$ вершин. Известно, что он может быть и выпуклым, и невыпуклым. А также может содержать дырки. Мы хотим получить его straight skeleton — $S(P_n)$.

Если две несмежные ранее стороны встретились, будем называть точку из скелета $edge\ event$.

А $split\ event$ — ребро разбивается на два новых, исходящих из точки преломления старого. Событие происходит на биссектрисе вогнутой вершины многоугольника. В этот момент стягиваемая область может разбиваться на две непересекающиеся области.

## Свойства Straight skeleton

>__Лемма 1.__ _$S(P_n)$ есть планарный граф. Более того — дерево._

$\triangleright$
<div style="padding-left:40px">
Планарность следует из процесса построения. 

То, что $S(P_n)$ — дерево, легко доказывается по индукции числа вершин в многоугольнике.

__База:__ треугольник $P_3$. В его $S(P_3)$ есть одна внутренняя вершина — точка пересечения биссектрис. Листьями будут вершины треугольника. Такой граф, очевидно, дерево.

__Переход:__ пусть для всех многоугольников с количеством вершин $i$, где $i < k$, $S(P_i)$ будет деревом. Тогда рассмотрим самый первый $event$ для многоугольника из $k$ вершин.
* $edge\ event$ $\Rightarrow$ новая вершина, которую соединим с инцидентными ребру вершинами, а также с какой-то вершиной из $S(P_{k-1})$. Полученный граф — дерево.
* $split\ event$ $\Rightarrow$ новая вершина соединяется с одной вершиной исходного многоугольника и с двумя вершинами из $S(P_i)$, где $i < k$. Полученный граф есть дерево.
</div>
$\triangleleft$

>__Лемма.__ _$S(P_n)$ cодержит $n$ граней, не более $n - 2$ внутренних вершин и не более $2n - 3$ ребер._

$\triangleright$
<div style="padding-left:40px">
Пусть грань $f(e)$ есть область между ребрами $S(P_n)$ и стороной $e$ из $P_n$. Даже если на ребре произошел $split\ event$, грань $f(e)$ не разделится. Построение грани завершится, когда ребро $e$ полностью стянется. Далее это ребро уже не появится. Следовательно, граней в $S(P_n)$ ровно столько, сколько сторон в $P_n$. То есть ровно $n$.

Внутренние вершины $S(P_n)$ имеют степень не меньше 3 — очевидно из определения. Если в одной вершине несколько событий, то степень будет только увеличиваться. Так как $S(P_n)$ имеет $n$ листьев, то внутренних вершин не больше $n - 2$. Так как $S(P_n)$ является деревом, то рёбер будет не более $2n - 3$.
</div>
$\triangleleft$

## Выпуклый многоугольник
Решим сначала более простую задачу. Пусть известно, что $P_n$ — выпуклый без дырок.

Суть алгоритма примерно в следующем: найдем точки пересечения биссектрис внутренних углов многоугольника $P_i$ для каждой вершины со всеми соседними вершиными, возьмем такую точку, в которой самый первый $edge\ event$. Добавим полученную вершину в $S(P_n)$. Соединим с вершинами ребра, которое изчезло в результате $edge\ event$. Затем перестроим $P_{i - 1}$, создав новую вершину, и подвинув все остальные вдоль биссектрис на одинаковое расстроение. Продолжим процесс, пока многоугольник не перейдет в треугольник. 

Рассмотрим более эффективную реализацию данной идеи. Для этого нам понадобится SLAV.

**SLAV** — структура, хранящая цикл всех вершин для внешней грани, а также цикл для каждой многоугольника и для всех многоугольников, возникающих в процессе построения $S(P_n)$. 

Если точнее, нам достаточно просто **LAV** — циклический список всех вершин многоугольника. В таком списке частично найденного $S(P_n)$ вершины имеют указатели на следующую и предыдущую вершины в порядке обхода контура, а также указатели на инцидентные рёбра.

### Алгоритм
Считаем, что многоугольник представлен рёбрами, направленными вдоль движения по контуру против часовой стрелки.

**Шаг 1:** Инициализация.
* Поместим все вершины многоугольника $v_1, v_2 \dots v_n$ в двусвязный циклический список в порядке обхода вдоль контура. Все вершины в LAV сейчас считаются активными.
* Для каждой вершины $v_i$ в LAV добавим указатели на инцидентные рёбра $e_{i - 1} = v_{i - 1}v_i$ и $e_i = v_iv_{i + 1}$. Также найдем луч биссектрисы $b_i$.
* Для каждой вершины $v_i$ найдем ближайшее пересечение биссектрисы $b_i$ с биссектрисами $b_{i-1}$ и $b_i$. Если пересечение существует, поместим его в приоритетную очередь $Q$ согласно расстроянию от точки пересечения до одного из ребер, инцидентных вершине $v_i$. Для каждой точки перечения $I_j$ будем хранить еще два указателя на вершины $v_x$ и $v_y$ — начала лучей биссектрис, которые пересекаются в точке $I_j$. Эти указатели понадобятся в будущем, когда нужно будет определять соответствующие вершинам рёбра $e_x$ и $e_y$.

**Шаг 2:** Выполняем следующие действия пока $Q$ не пуста.
* Извлечем точку пересечения $I$ из $Q$.
* Если $v_x$ и $v_y$, соотвествующие $I$, помечены, как обработанные, переходим к следующей итерации шага 2, так как это означает, что ребро между данными вершинами полностью стянулось. Проверку необходимо делать из-за того, что мы могли поместить в очередь обработанные вершины в момент получения новых $event$. 
    **NB.** Если осталось только 3 вершины $v_x, v_y, v_z$, то добавим в $S(P)$ рёбра $Iv_x, Iv_y, Iv_z$. 
* Добавим в $S(P)$ рёбра $Iv_x$ и $Iv_y$.
* Модифицируем LAV:
    * пометим $v_x$ и $v_y$ как обработанные;
    * создадим новую вершину $v$ в точке пересечения $I$;
    * добавим вершину $v$ в LAV, то есть между предшествующим $v_a$ и следующим за $v_b$ узлами;
    * добавим вершине $v$ указатели на соответствующие рёбра $e_x$ и $e_y$.
* Посчитаем все необходимые дополнительные величины для вершины $v$:
    * луч биссектрисы $b$ между рёбрами $e_x$ и $e_y$;
    * точки пересечения биссектрисы $b$ с биссектрисами вершин, соседних к $v$ в LAV;
    * сохраним ближайшие точки пересечения в $Q$. Ключом точки пересечения в $Q$ будет расстрояние до стянутого ребра $e_i$.

### Асимптотика
Так как на каждой итерации алгоритма в очередь кладется константное число элементов, а итераций не больше $n$, время работы — $O(n \log{n})$.

## Невыпуклый многоугольник. Felkel's algorithm
Основной принцип для невыпуклых полигонов такой же. Только теперь мы хотим хранить для вершины пометку, какое именно событие в ней произошло: $split\ event$ или $edge\ event$.

Если представить процесс стягивания в объеме, как крышу, у которой основание это наш многоугольник, а рёбра — $S(P)$, то пересечение крыши и плоскости, параллельной основанию на высоте $t$ есть слой стягивания в момент времени $t$. Область полигона разбивается на несколько частей. Каждой части соотвествует свой  LAV. Поэтому нам нужен SLAV.

Наличие невыпуклой вершины может привести к разделению внутренней области. Однако так же невыпуклая вершина может участвовать и в $edge\ event$. Все $edge\ event$ обрабатываются так же, как и в алгоритме для выпуклого многоугольника. 

Нам же интересен случай, когда невыпуклая точка $b$ порождает $split\ event$. Рассмотри, как находить координаты этой точки $b$. Формально $b$ можно охарактеризовать как точку, находящуюся на равном расстоянии от противолежащего ребра и прямых, содержащих рёбра невыпуклой вершины. Задача состоит в том, чтобы найти это самое противолежащее ребро. Для этого нам необходимо проверять, что точка $b$ лежит в треугольнике, ограниченном противолежащим ребром и биссектрисами $b_i$ и $b_{i+1}$, идущими из вершин, инцидентных противолежащему ребру $e_i$ (треугольник может быть "бесконечным").

Координаты возможной точки кандидата $b_i$ вычисляются следующим образом: это точка пересечения биссектрисы вершины $v$ и биссектрисы угла, который образуется в точке пересечения прямой, содержащей одно из рёбер, инцидентных $v$, и прямой, содержащей противолежащее ребро $e_i$. Все такие точки пересечения $b_i$ нужно поместить в приоритетную очередь.

Когда происходит работа с точкой $b$, которая порождает $split\ event$, необходимо разбить соответствующий полигон на две части, что соответсвует разделению LAV данного полигона на два списка. И в каждый новый список нужно вставить копию вершины $v$, образующейся в точке пересечения $I$. Обе новые вершины-копии $v_1$ и $v_2$ указывают на разделяющее ребро $e_i$.

### Частный случай множественных split event'ов на одном ребре
Уже должно было стать понятно, что алгоритм не строит промежуточного представления straight skeleton, а работает исключительно с рёбрами исходного полигона. Это приводит к ситуации, когда одно ребро является общим для нескольких новых полигонов в промежуточном представлении (то есть одно ребро разбивается несколько раз), образовавшихся после разделения старого полигона. В случае, когда ребро уже разбито, и происходит следующий за ним $event$, необходимо правильно определить концы противолежащего ребра (то есть вершины/узы, которые активны в текущем уровне конструирования крыши.

Чтобы решить эту проблему, следует хранить $split\ event$ как 3 вершины -- невыпуклая вершина и две вершины противолежащего ребра. Дополнительно нужно хранить ассоциативный массив из пары вершин в ребро для этих вершин. Тогда в момент разделения ребра необходимо удалить это ребро из ассоциативного массива и поместить туда два новых ребра, которые будут ссылаться на исходное ребро $e_i$.

Но в очереди могло быть событие по трём вершинам исходного ребра, а после разделения этого ребра уже нет (например, такое может произойти, если с ребром одновременно сталкивается несколько невыпуклых вершин, лежащих на параллельной этому ребру прямой). Посмотрев в ассоциативный массив, можно обнаружить, что такой пары вершин нет. Однако нам нужно всё же получить по ребру требуемую пару вершин. Для этого можно хранить ещё один ассоциативный массив из пар в рёбра, только из него уже не удалять старые пары. Тогда станет возможным получение по паре вершин ребра, а потом по ребру можно будет получить все актуальные пары вершин, соотвествующих этому ребру, и добавить $split\ event$ с нужной парой вершин.

### Алгоритм
**Шаг 1:** Инициализация.
* Положим все вершины в LAV, как это делается для выпуклых многоугольников, а потом LAV поместить в SLAV.
* Найдем биссектрисы как в случае с выпуклым многоугольником.
* Для каждой биссектрисы выпуклой вершины найдем ближайшую точку пересечения с биссектрисой соседней вершины, а для невыпуклых вершин найдем также точки пересечения с противолежащими рёбрами (как это описывалось раньше) и положим в приоритетную очередь $Q$ ближайшую точку пересечения $I$. Будем также с этой точкой хранить её тип -- $edge\ event$ или $split\ event$.

**Шаг 2:** Выполняем следующие действия пока $Q$ не пуста.
* Извлечем точку пересечения $I$ из приоритетной очереди. Если она имеет тип $edge\ event$, то её надо обработать так же, как в алгоритме для выпуклого многоугольника на втором шаге в пунктах 2-6. Иначе выполняем следующие пункты.
* Если точка пересечения указывает на уже обработанные вершины, то продолжить со следующей итерации шага 2, как в случае с выпуклым полигоном. По этой причине мы не будем обрабатывать лишние $split\ event$'ы, хотя могли бы их добавить в очередь.
* Нужно сделать примерно то же, что в аналогичном пункте в алгоритме для выпуклых (шаг 2, пункт 3). Только на этом цикл не завершится, а продолжается с новой итерации, так как многоугольник мог разделиться на несколько частей, и, возможно, мы обработали лишь один подполигон, и он не является последним.
* Добавим в $S(P)$ ребро $Iv$, где точка $I$ указывает на вершину $v$. Для $split\ event$'ов точки пересечения указывают ровно на одну вершину в SLAV.
* Модифицируем теперь SLAV.
    * Пометим вершину $v$ как обработанную.
    * Создадим две новые вершины $v_1$ и $v_2$ с одинаковыми координатами точки пересечения $I$.
    * Найдем для каждой вершины $v_1$ и $v_2$ противолежащее ребро в своём подполигоне.
    * Разделим LAV с вершиной $v$ на две части, вставим в части вершины $v_1$ и $v_2$, а затем добавим в SLAV. Вершина $v_1$ будет следующей для предыдущего к $v$ узла в LAV и предыдущей для конца противолежащего ребра. Аналогично для вершины $v_2$.
    * Удалим из ассоциативного массива пару вершин ребра $e_i$ и поместим две новые пары, в одной из которых будет вершина $v_1$  и конец ребра $e_i$, а в другой -- начало $e_i$ и вершина $v_2$.
    * Добавим указатели на соответсвующие рёбра вершинам $v_1$ и $v_2$.
* Для обеих вершин $v_1$ и $v_2$ сделаем следующие пункты.
    * Найдём биссектрисы между рёбрами, на которые эти вершины указывают в предыдущем шаге.
    * Найдём все события точек пересечения, как в первом шаге 3 пунтке (тут могут получиться события обоих типов).
    * Сохраним в $Q$ точки пересечения, отвечающие найденным событиям.

### Асимптотика
Обработка $edge\ event$ выполняется с такой же асимптотикой, как и в алгоритме для выпуклых многоугольников. Но из-за того, что в очередь кладутся все возможнные точки, в которых произошли $split\ event$, в ней может оказаться $O(n^2)$ элементов. Хотя обработка $split\ event$ может занять линейное время от числа вершин (если новая вершина в точке пересечения осталась невыпуклой), это произойдет только для $split\ event$, которые создают новые вершины $S(P)$, а их $O(n)$ по доказанной лемме. Остальные $split\ event$ обработаются за $O(1)$ во 2 шаге 2 пункте, поэтому итоговая асимптотика $O(n^2 \log{n})$.

## Невыпуклый многоугольник с дырками
Алгоритм для невыпуклого многоугольника работает и с многоугольника, содержащим дырки, если они ориентированы по часовой стрелке, чтобы внутренняя область многоугольника лежала слева от рёбер. И в самом начале алгоритма каждый замкнутрый контур помещается в свой LAV в множестве SLAV.

## Особенности реализации и частные случаи
Приведенный алгоритм отлично работает на произвольных случайных полигонах, в которых возникают только простые события. На практике бывает что-то чуть менее тривиальное -- совпание многих  $edge\ event$, многих $split\ event$, или же в одной точке события сразу нескольких типов. Также параллельные ребра могут многократно накладываться друг на друга.

### Параллельные противоположные ребра
Как только параллельные рёбра становятся соседними перед событием, нужно проверить, что после произошеднего события они соединятся потом в одно ребро. Если в LAV осталось только два параллельных ребра, то мы удаляем их из SLAV.

### Параллельные "соседние" рёбра
Другой интересный случай возникает, когда несоседние параллельные рёбра становятся соседнии после исчезновения рёбер между ними. Такая проблема называется **PCE** *(parallel consecutive edge problem)*. Есть несколько способов решения.
* **Separate rule** -- согласно этому правилу два ребра рассматриваются отдельно. Тогда верно утверждение, что каждому ребру соответствует ровно одна грань. И в этом случае можно считать, что новая вершина на стыке двух рёбер движется перпендикулярно рёбрам.
* **Merge rule** -- рёбра в таком случае объединяются в одно новое ребро.

Отличить этот случай от предыдущего можно, посмотрев на ориентацию двух рёбер. Если они направлены в одну сторону, то это PCE, если в противоположную, то разбираемся как в предыдущем случае.

### Множественные события в одной точке
Первая проблема, возникающая в этом случае -- точное определение того, что несколько событий произошли в одной точке. Для определения совпадения нескольких событий в одной точке можно поступать приближённо: вводить с каждой точкой $\epsilon$-окрестность и смотреть, не попали ли другие точки в эту окрестность, или использовать более точную арифметику. В данном случае недостаточно использовать интервальную арифметику или даже рациональную арифметику. Потому что даже если координаты точек задаются абсолютно точно, то для подсчёта радиуса вписанной окружности необходимо уметь извлекать корни.

Чтобы научиться разбираться с такими случаями в алгоритме, когда мы уже поняли, что в одной точке будет несколько событий, введём понятие **обобщённого события пересечения** *(GIE, generalized intersection event)*. Для начала введём понятие цепочек рёбер, которые вовлечены в событие -- рёбра, сталкивающиеся в данном событии. Эти цепи упорядочим согласно направлению рёбер.
Мы можем также упорядочить сами цепи вокруг точки события, объединив эти цепи в один циклический список. Следовательно, событие получается как бы окружено списком рёбер, которые участвуют в нём. При этом никакие другие рёбра не участвуют. Можно заметить, что соседние рёбра в списке из изначально разных цепей становятся потом соседними в LAV.

#### Алгоритм обработки GIE

* **Внутри цепи**. В каждой цепи удаляем внутренние рёбра (кроме первого и последнего). Это соответствует тому, что исчезает несколько рёбер, участвующих в одном $edge event$. После этого шага остаются цепи только длин 1 или 2.
* **Цепи из одного звена.** Цепи длины 1 разбиваются в точке события. Это соответствует простому $split event$. Теперь все цепи имеют длину ровно 2.
* **Межцепной.** Объединяем соседние цепи, соответствующие одному событию, в циклический список, соединяя конец одной цепи с началом следующей и так далее. Каждая цепь разбивается в середине. Образуются новые списки длины 2.
* **Циклы из двух рёбер.** Списки LAV длины 2, состоящие из двух параллельных рёбер, то есть ограничивающие полигон нулевой площади, удаляются из SLAV.
* **PCE.** Разбираемся с PCE согласно принятому нами правилу решения -- правилу слияния или правилу разделения.

В реализации это будет выглядеть следующим образом. Можно посмотреть, что сейчас лежит в голове приоритетной очереди, и доставать события, пока они происходят в одной точке. Затем разбить эти события на цепочки и выполнять шаги из алгоритма выше.

## Другие алгоритмы и Открытые реализации
Приведенный в конспекте алгоритм реализован Fernando Cacciola, который исправил все ошибки в статье P. Felkel. И этот алгоритм используется в открытой библиотеке вычислительной геометрии CGAL. Более того, он является одной из немногих открытых реализаций построения straight skeleton. Но данный алгоритм достаточно медленный для решения практических задач. В реальной жизни используют его модификации или более сложные алгоритмы.

Существует простой в понимании и реализации алгоритм для построения straigt skeleton на основе триангуляции, который работает за время $O(n^3 \log{n})$. Aichholzer смог обобщить этот алгоритм для построения straigt skeleton произвольного планарного графа. Также автором в оригинальной статье был представлен алгоритм построения данной структуры, базирующийся на понятии **волнового фронта** *(wavefront)*. Этот алгоритм может быть реализован за время $O(n^3)$ с использованием $O(n)$ памяти либо с использованием приоритетной очереди за время $O(n^2 \log{n})$ и $O(n^2)$ памяти. Известен алгоритм построения straight skeleton для монотонных полигонов за время $O(n \log{n})$ с использованием $O(n)$ памяти. Также существует более эффективный алгоритм на основе мотографов, который придумали Stefan Huber и Martin Held. Их реализация называется **STALGO**, и она доступна на их сайте.
